# Generador de listas de correos para envío y notificaciones invitados ECCI
Notebook para la generación de listas de correos electrónicos para envío masivo y periódico de notificaciones a los invitados del Encuentro Ciudades y Culturas en Iberoamérica. Se realiza lo siguiente: Verificación de estados de confirmación, consolidación de listas, verificación de emails, verificación de envíos previos. Generación de archivo excel con lista de correos para envío.
- Mauricio Ojeda Pepinosa
- javier.ojeda@scrd.gov.co
- 2025-09-15
- Sistemas de Información y Narrativa

In [16]:
import pandas as pd

In [17]:
# Código del grupo o lote de envío de correos
mailing_group = '20250915-bienvenida'

In [18]:
filename = 'BDD_InvitadxsACI_2025.xlsx'
df_internacionales = pd.read_excel(filename, sheet_name='Internacionales')
df_colombia = pd.read_excel(filename, sheet_name='Colombia')
df_bogota = pd.read_excel(filename, sheet_name='Bogotá')
df_secretcolombia = pd.read_excel(filename, sheet_name='SecretColombia')

In [19]:
df_internacionales['lista'] = 'Internacionales'
df_colombia['lista'] = 'Colombia'
df_bogota['lista'] = 'Bogotá'
df_secretcolombia['lista'] = 'SecretColombia'

In [20]:
df_internacionales = df_internacionales[df_internacionales['Estado de confirmación'] == 'Confirma asistencia']
df_colombia = df_colombia[df_colombia['Estado de confirmación'] == 'Confirma asistencia']
df_bogota = df_bogota[df_bogota['Estado de confirmación'] == 'Confirma asistencia']
df_secretcolombia = df_secretcolombia[df_secretcolombia['Estado de confirmación'] == 'Confirma asistencia']

In [21]:
# En df_internacionales, concatenar Email y Email registro, separando con comas, debe ser tipo string
df_internacionales['Email'] = (df_internacionales['Email'].fillna('') + ', ' + df_internacionales['Email registro'].fillna('')).astype(str)
df_colombia['Email'] = (df_colombia['Email'].fillna('') + ', ' + df_colombia['Email registro'].fillna('')).astype(str)
df_bogota['Email'] = (df_bogota['Email'].fillna('') + ', ' + df_bogota['Email registro'].fillna('')).astype(str)
df_secretcolombia['Email'] = (df_secretcolombia['Email'].fillna('') + ', ' + df_secretcolombia['Email registro'].fillna('')).astype(str)

In [22]:
columnas = ['ID', 'Nombre completo', 'Email', 'lista']
df_internacionales = df_internacionales[columnas]
df_colombia = df_colombia[columnas]
df_bogota = df_bogota[columnas]
df_secretcolombia = df_secretcolombia[columnas]

In [24]:
# Combinar los tres dataframes, garantizando que la columna "Email" sea string
df_internacionales['Email'] = df_internacionales['Email'].astype(str)
df_colombia['Email'] = df_colombia['Email'].astype(str)
df_bogota['Email'] = df_bogota['Email'].astype(str)
df_secretcolombia['Email'] = df_secretcolombia['Email'].astype(str)

df_invitados = df_internacionales.merge(df_colombia, on=['ID', 'Nombre completo', 'Email', 'lista'], how='outer')
df_invitados = df_invitados.merge(df_bogota, on=['ID', 'Nombre completo', 'Email', 'lista'], how='outer')
df_invitados = df_invitados.merge(df_secretcolombia, on=['ID', 'Nombre completo', 'Email', 'lista'], how='outer')

In [26]:
# Quitar los que tiene Email vacío
df_invitados = df_invitados[df_invitados['Email'].notna()]

In [27]:
# Duplicar las filas para los invitados que tienen más de un Email, separados por coma#
# Cuando un invitado tiene más de un email en la columna Email, se deben crear un registro adicional en el DataFrame
df_invitados = df_invitados.assign(Email=df_invitados['Email'].str.split(',')).explode('Email')


In [28]:
# Quitar repetidos
df_invitados = df_invitados.drop_duplicates(subset=['Email'])
# Quitar espacios en blanco en la columna Email
df_invitados['Email'] = df_invitados['Email'].str.strip()
# Quitar las filas donde no hay Email
df_invitados = df_invitados[df_invitados['Email'] != '']

In [29]:
df_invitados['envio'] = mailing_group
df_invitados.shape

(446, 5)

In [30]:
with pd.ExcelWriter('users_upload.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_invitados.to_excel(writer, sheet_name='mailing', index=False)